Going to run the GNB model in order to get a good baseline for performace
1. Control vs. All Park
1. PD vs. MSA/PSP
1. MSA vs. PD/PSP
1. PDP vs. PD/MSA

## Imports and Function Definitions

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, train_test_split, LeaveOneOut, GridSearchCV
from sklearn.naive_bayes import GaussianNB

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

def grid_search_optimization(model, tuned_parameters, X, y, Xh, yh, cv=5, scoring='accuracy', verbose=False):
    print("# Tuning hyper-parameters for %s" %scoring)
    print()

    clf = GridSearchCV(model, tuned_parameters, cv=cv, n_jobs = -1, scoring=scoring, verbose=1)
    clf.fit(X, y)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    if verbose:
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

    print("Detailed classification report (holdout):")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = yh, clf.predict(Xh)
    print(classification_report(y_true, y_pred))
    print()
    
    return clf

def group_classes(data, grouping):
        classes_to_keep = grouping.keys()
        data_to_keep = data.loc[data['GroupID'].isin(classes_to_keep)]
        classes_to_change = {k:grouping[k] for k in classes_to_keep if k!= grouping[k]}
        return data_to_keep.replace(classes_to_change)

In [12]:
# parameters and ranges to plot
param_grid = {
#     "classifier__C": np.logspace(-5, 1, 20),
#     "PCA__n_components": range(1,113,10)
    #"fss__k" : range(1,115,3)
}

clf = Pipeline([
#     ('Norm', Normalizer()),
    #("Oversample", RandomOverSampler()),
#     ('PCA', PCA()),
#     ('fss',SelectKBest()),
#     ('Scaler', StandardScaler()),
    ('classifier', GaussianNB())
])

## 1. Control vs. PD/MSA/PSP

In [3]:
# Get the data
data1 = pd.read_excel('training_data.xlsx')
data1 = group_classes(data1, {0:0, 1:1, 2:1, 3:1})

y1 = data1['GroupID']
X1 = data1.drop(['GroupID'], axis=1)

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, y1, test_size=0.35, random_state=42)

best1 = grid_search_optimization(clf, param_grid, X_train1, Y_train1, X_test1, Y_test1, cv=20, scoring='f1_micro')

# Tuning hyper-parameters for f1_micro

Fitting 20 folds for each of 112 candidates, totalling 2240 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 770 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 1895 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 2225 out of 2240 | elapsed:   29.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed:   29.7s finished


Best parameters set found on development set:

{'PCA__n_components': 1}

Detailed classification report (holdout):

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        95
          1       0.70      1.00      0.83       227

avg / total       0.50      0.70      0.58       322




C:\Users\jtbricker\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 2. PD vs MSA/PSP

In [4]:
# Get the data
data2 = pd.read_excel('training_data.xlsx')
data2 = group_classes(data2, {1:0, 2:1, 3:1})

y2 = data2['GroupID']
X2 = data2.drop(['GroupID'], axis=1)

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, y2, test_size=0.35, random_state=42)

best2 = grid_search_optimization(clf, param_grid, X_train2, Y_train2, X_test2, Y_test2, cv=20, scoring='f1_micro')

# Tuning hyper-parameters for f1_micro

Fitting 20 folds for each of 112 candidates, totalling 2240 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 770 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed:   17.2s finished


Best parameters set found on development set:

{'PCA__n_components': 39}

Detailed classification report (holdout):

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.87      0.91      0.89       164
          1       0.74      0.65      0.69        66

avg / total       0.83      0.83      0.83       230




## 3. MSA vs PD/PSP

In [13]:
# Get the data
data3 = pd.read_excel('training_data.xlsx')
data3 = group_classes(data3, {1:0, 3:0, 2:1})

y3 = data3['GroupID']
X3 = data3.drop(['GroupID'], axis=1)

X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3, y3, test_size=0.35, random_state=42)

best3 = grid_search_optimization(clf, param_grid, X_train3, Y_train3, X_test3, Y_test3, cv=20, scoring='f1_micro')

# Tuning hyper-parameters for f1_micro

Fitting 20 folds for each of 1 candidates, totalling 20 fits
Best parameters set found on development set:

{}

Detailed classification report (holdout):

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.93      0.85      0.89       208
          1       0.21      0.36      0.26        22

avg / total       0.86      0.80      0.83       230




[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    6.3s finished


## 4. PSP vs PD/MSA

In [6]:
# Get the data
data4 = pd.read_excel('training_data.xlsx')
data4 = group_classes(data4, {1:0, 2:0, 3:1})

y4 = data4['GroupID']
X4 = data4.drop(['GroupID'], axis=1)

X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4, y4, test_size=0.35, random_state=42)

best4 = grid_search_optimization(clf, param_grid, X_train4, Y_train4, X_test4, Y_test4, cv=20, scoring='f1_micro')

# Tuning hyper-parameters for f1_micro

Fitting 20 folds for each of 112 candidates, totalling 2240 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 770 tasks      | elapsed:   10.4s


Best parameters set found on development set:

{'PCA__n_components': 1}

Detailed classification report (holdout):

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.90      0.97      0.93       186
          1       0.82      0.52      0.64        44

avg / total       0.88      0.89      0.88       230




[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed:   17.6s finished


In [ ]:
best4.best_estimator_.score(X_test4, Y_test4)